In [1]:
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.models as models

import sys
import math

In [2]:
class Bottleneck(nn.Module):
    def __init__(self, nChannels,growthRate):
        super(Bottleneck,self).__init__()
        interChannels = 4*growthRate
        self.bn1=nn.BatchNorm2d(nChannels)
        self.conv1=nn.Conv2d(nChannels, interChannels,kernel_size=1,
                            bias=False)
        self.bn2=nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)
    def forward(self,x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

In [3]:
class Transition(nn.Module): # 반으로 줄여주는 작업임.
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out

In [4]:
class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3 # 만약 depth가 100이라면 32
        if bottleneck:
            nDenseBlocks //= 2 # Bottleneck이 있다면  16

        nChannels = 2*growthRate # growthRate 12
        self.conv1 = nn.Conv2d(3, nChannels, kernel_size=3, padding=1,
                               bias=False)
        # 1, 24, 50, 50
        
        #nDenseBlock => 16(Bottleneck)
        #nChannel => 24
        #growthRath => 12
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        # 1, 216, 50, 50
        # 12*18 => 12 == growthRath 
        nChannels += nDenseBlocks*growthRate
        # 24 + 192 =>nChannels
        #print(nChannels)
        nOutChannels = int(math.floor(nChannels*reduction)) #math.floor는 소수점 내림.
        # nOutChannels => 108
        self.trans1 = Transition(nChannels, nOutChannels)
        # 1, 108, 25, 25 반반으로 짜름

        nChannels = nOutChannels
        # 1, 108, 25, 25 => x 
        # nChannels => 108
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        # 1, 300, 25, 25
        # 108 + 12*16 = > 300
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)
        # 1, 150, 12 ,12

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        # 1, 342, 12, 12
        # 192를 계속 더해줌 Tip!
        # 150+192=>342
        nChannels += nDenseBlocks*growthRate
        # print(nChannels)
        # nChannels => 342
        

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        # layer의 초기화 
        for m in self.modules():
            if isinstance(m, nn.Conv2d): # m이 nn.Conv2d이면
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # n=> size
                m.weight.data.normal_(0, math.sqrt(2. / n))
                #zero-mean Gaussian distribution Standard deviation
                
                #print(m.weight.data)
                # 가중치 초기화
            elif isinstance(m, nn.BatchNorm2d): # m이 nn.BatchNorm2d이면
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                # bias => 0
            elif isinstance(m, nn.Linear): # m이 nn.Linear이면
                m.bias.data.zero_()
                # bias=> 0
        # xavier method => initial differnece thesis ReLu/PReLU 제외
        # delving deep into rectifiers: surpassing human level performance on imagenet classification

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        # 1, 24, 50, 50
        out = self.trans1(self.dense1(out))
        # 1, 216, 50, 50
        # 1, 108, 25, 25
        out = self.trans2(self.dense2(out))
        # 1, 300, 25, 25
        # 1, 150, 12 ,12
        out = self.dense3(out)
        # 1, 342, 12 ,12
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
        # avg_pool2d out => 1, 342, 1, 1
        out = F.log_softmax(self.fc(out))
        return out

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [6]:
input_data=np.load("input_data_emotion.npy")
label=np.load("label.np.npy")

In [7]:
label=np.array(label)
input_data= (input_data*(1/256)-0.5)/0.5

In [8]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [9]:
input_data=input_data.transpose(1,3)

In [10]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [11]:
trainset,valset=torch.utils.data.random_split(dataset,[30887,5000])
trainset,testset=torch.utils.data.random_split(trainset,[25887,5000])

partition={'train':trainset,'val':valset,'test':testset}

In [12]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=32,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [13]:
def test(net):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=32, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [14]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=32, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [15]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [16]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net = DenseNet(growthRate=12, depth=100, reduction=0.5,
                            bottleneck=True, nClasses=7)
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

  0%|          | 1/809 [00:00<01:31,  8.79it/s]

time : 92.67, train_loss : 1.55, train_acc : 38.30,val_acc : 49.14,val_loss : 1.34


  0%|          | 1/809 [00:00<01:32,  8.78it/s]

time : 92.94, train_loss : 1.23, train_acc : 53.06,val_acc : 51.46,val_loss : 1.30


  0%|          | 1/809 [00:00<01:33,  8.66it/s]

...

time : 93.34, train_loss : 0.07, train_acc : 97.73,val_acc : 61.52,val_loss : 2.48


  0%|          | 1/809 [00:00<01:33,  8.65it/s]

time : 93.34, train_loss : 0.07, train_acc : 97.83,val_acc : 62.34,val_loss : 2.38


100%|██████████| 157/157 [00:03<00:00, 41.31it/s]

time : 93.37, train_loss : 0.06, train_acc : 97.98,val_acc : 61.60,val_loss : 2.40


In [17]:
np.save("/home/lab/양창희/Emotion Project/test_list_file/Dense_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Dense_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Dense_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Dense_val_acc.npy",val_acc)